In [ ]:
from datetime import datetime
#from datetime import datetime, timedelta
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sql_expr, lit
import sys

# ---------- Configuration ----------
folder_path = datetime.utcnow().strftime("%Y/%b/%d/").title()  # e.g. 2025/Sep/03/
print("📌 Using folder path:", folder_path)

#tomorrow = datetime.utcnow() + timedelta(days=-1)
#folder_path = tomorrow.strftime("%Y/%b/%d/").title()
#print("📌 Using folder path:", folder_path)



connection_parameters = {
    "account": "MJ13681.ap-southeast-1",
    "user": "mzs1988",
    "password": "Datalabs@193001",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "DEV",
    "schema": "DATASCIENCE"
}

In [ ]:
# Entities and patterns
entities = {
    "customers": {
        "bronze_table": "bronze_customers",
        "silver_table": "silver_dlt_customers",
        "pattern": r".*Customers_.*?/part-.*\.parquet",
        "required_cols": ["customer_id", "customer_name"],
        "casts": {
            "customer_id": "int",
            "customer_name": "string",
            "start_date": "timestamp"
        }
    },
    "products": {
        "bronze_table": "bronze_products",
        "silver_table": "silver_dlt_products",
        "pattern": r".*Products_.*?/part-.*\.parquet",
        "required_cols": ["product_id", "product_name"],
        "casts": {
            "product_id": "string",
            "product_name": "string",
            "start_date": "timestamp"
        }
    },
    "orders": {
        "bronze_table": "bronze_orders",
        "silver_table": "silver_dlt_orders",
        "pattern": r".*Orders_.*?/part-.*\.parquet",
        "required_cols": ["order_id", "customer_id", "product_id"],
        "casts": {
            "order_id": "string",
            "customer_id": "int",
            "product_id": "string",
            "start_date": "timestamp"
        }
    }
}

# ---------- Create Snowflake session ----------
session = Session.builder.configs(connection_parameters).create()
print("✅ Snowflake session created")

# helper SQL expressions
#now_expr = "TO_TIMESTAMP_NTZ(TO_CHAR(CURRENT_TIMESTAMP(), 'YYYY-MM-DD HH24:MI:SS'))"
now_expr = "TO_TIMESTAMP_NTZ(TO_CHAR(CONVERT_TIMEZONE('UTC', CURRENT_TIMESTAMP()), 'YYYY-MM-DD HH24:MI:SS'))"
open_end_date_literal = "TO_TIMESTAMP_NTZ('2099-12-31 23:59:59')"

def stage_has_files(entity_name):
    """
    Use LIST @stage/path to check if any object exists for today's folder for the entity.
    Returns True if at least one object found.
    """
    stage_path = f"@bronze_csv/{entity_name}/Full_load/csv_files/{folder_path}"
    try:
        df = session.sql(f"LIST {stage_path};")
        rows = df.collect()
        return len(rows) > 0
    except Exception as e:
        # If LIST fails (e.g., path doesn't exist) treat as no files
        print(f"⚠️ LIST failed for {stage_path}: {str(e)}")
        return False

In [ ]:
# ---------- Step 1: Overwrite bronze tables if today's parquet present ----------
for entity, meta in entities.items():
    bronze_table = meta["bronze_table"]
    stage_subpath = f"@bronze_csv/{entity}/Full_load/csv_files/{folder_path}"
    pattern = meta["pattern"]

    has_files = stage_has_files(entity)
    if has_files:
        print(f"📥 Found parquet files for {entity} at {stage_subpath} — will overwrite {bronze_table}.")
        # Truncate and load from stage into bronze table
        # TRUNCATE then COPY INTO table
        try:
            session.sql(f"TRUNCATE TABLE {bronze_table};").collect()
            copy_sql = f"""
            COPY INTO {bronze_table}
            FROM {stage_subpath}
            FILE_FORMAT = (FORMAT_NAME = my_parquet_format)
            MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
            PATTERN = '{pattern}';
            """
            
            session.sql(copy_sql).collect()
            print(f"✅ Overwrote table {bronze_table} from stage {stage_subpath}")
        except Exception as e:
            print(f"❌ Error loading {entity} into {bronze_table}: {e}")
            session.close()
            raise
    else:
        print(f"ℹ️ No parquet files found for {entity} at {stage_subpath}. Keeping existing {bronze_table} data.")


In [ ]:
df = session.table("bronze_products")
df.show(15)


In [ ]:
# ---------- Step 2: Cleaning bronze (remove nulls, dedupe, cast, add load_date/end_date) ----------
# We'll create temporary tables bronze_{entity}_clean to use in merge logic.
for entity, meta in entities.items():
    bronze_table = meta["bronze_table"]
    clean_table = f"{bronze_table}_clean"
    required_cols = meta["required_cols"]
    casts = meta["casts"]

    # Build WHERE clause for non-null required cols
    not_null_conditions = " AND ".join([f"{col} IS NOT NULL" for col in required_cols])

    # Build CAST expressions (use SELECT ... )
    select_expressions = []
    for col_name, dtype in casts.items():
        if dtype.lower() == "int":
            select_expressions.append(f"TRY_TO_NUMBER({col_name}) AS {col_name}")
        elif dtype.lower() == "timestamp":
            # Ensure start_date not in the future
            select_expressions.append(f"{col_name} AS {col_name}")
        else:
            # string → trim
            select_expressions.append(f"TRIM({col_name}) AS {col_name}")

    select_clause = ",\n        ".join(select_expressions)

    # Create cleaned temporary table
    create_clean_sql = f"""
    CREATE OR REPLACE TEMPORARY TABLE {clean_table} AS
    SELECT DISTINCT
        {select_clause},
        {now_expr} AS load_date,
        {open_end_date_literal} AS end_date
    FROM {bronze_table}
    WHERE {not_null_conditions}
      AND start_date IS NOT NULL
      AND start_date <= {now_expr}
    ;
    """
    session.sql(create_clean_sql).collect()
    print(f"✅ Created cleaned table: {clean_table}")

In [ ]:
df = session.table("bronze_products_clean").order_by("product_id")
df.show(15)

In [ ]:
# ---------- Step 3: Merge cleaned bronze into silver_dlt tables (SCD Type 2) ----------
for entity, meta in entities.items():
    bronze_clean = f"{meta['bronze_table']}_clean"
    silver_table = meta['silver_table']

    # Determine key and columns to compare
    if entity == "customers":
        key = "customer_id"
        compare_cols = ["customer_name", "start_date"]
    elif entity == "products":
        key = "product_id"
        compare_cols = ["product_name", "start_date"]
    else:  # orders
        key = "order_id"
        compare_cols = ["customer_id", "product_id", "start_date"]

    # Use IS DISTINCT FROM to safely detect differences (handles NULLs)
    change_conditions = " OR ".join([f"(tgt.{c} IS DISTINCT FROM src.{c})" for c in compare_cols])

    # -----------------------
    # 1) EXPIRE old active rows for records that have changed
    #    Set old row's end_date = src.start_date (truncate to seconds)
    # -----------------------
    expire_changed_sql = f"""
    UPDATE {silver_table} tgt
    SET end_date = DATE_TRUNC('SECOND', src.start_date)
    FROM {bronze_clean} src
    WHERE tgt.{key} = src.{key}
      AND tgt.end_date = {open_end_date_literal}
      AND ({change_conditions});
    """
    session.sql(expire_changed_sql).collect()
    print(f"✅ Expired old active rows in {silver_table} for changed records (end_date set to new start_date)")

    # -----------------------
    # 2) INSERT new rows:
    #    Insert any src rows that do not currently have an active target row.
    #    This inserts brand-new keys AND also inserts new rows for updated keys
    #    because the previous step expired the active row for updated keys.
    # -----------------------
    insert_new_sql = f"""
    INSERT INTO {silver_table} ({', '.join([key] + compare_cols + ['load_date', 'end_date'])})
    SELECT
      src.{key},
      {', '.join(['src.' + c for c in compare_cols])},
      DATE_TRUNC('SECOND', CONVERT_TIMEZONE('UTC', CURRENT_TIMESTAMP())::TIMESTAMP_NTZ) AS load_date,
      TO_TIMESTAMP_NTZ('2099-12-31 23:59:59') AS end_date
    FROM {bronze_clean} src
    WHERE NOT EXISTS (
      SELECT 1 FROM {silver_table} tgt
      WHERE tgt.{key} = src.{key}
        AND tgt.end_date = {open_end_date_literal}
    );
    """
    session.sql(insert_new_sql).collect()
    print(f"✅ Inserted new/updated rows into {silver_table} (load_date = UTC now, end_date = 2099-12-31)")

    # -----------------------
    # 3) DELETES: expire active silver rows missing from bronze_clean
    #    Mark them deleted by setting end_date = current load timestamp (seconds precision, UTC)
    # -----------------------
    expire_deleted_sql = f"""
    UPDATE {silver_table}
    SET end_date = DATE_TRUNC('SECOND', CONVERT_TIMEZONE('UTC', CURRENT_TIMESTAMP())::TIMESTAMP_NTZ)
    WHERE end_date = {open_end_date_literal}
      AND {key} NOT IN (SELECT {key} FROM {bronze_clean});
    """
    session.sql(expire_deleted_sql).collect()
    print(f"✅ Expired rows in {silver_table} that are missing from today's bronze (logical deletes)")

print("✅ Step 3 complete: updates/insertions/deletes applied (SCD2 semantics).")


In [ ]:
session.table("silver_dlt_orders").order_by("order_id").show(15)



In [ ]:
# ---------- Step 4: Export updated silver tables back to S3 (parquet) ----------
# Stage silver_csv should point to s3://datascience-output-bucket/silver/
for entity, meta in entities.items():
    silver_table = meta['silver_table']
    # Use entity capitalized name in file if you want (Customers/Products/Orders)
    capital_entity = entity.capitalize()
    copy_out_sql = f"""
    COPY INTO @silver_csv/{entity}/Full_load/csv_files/{folder_path}{capital_entity}.parquet
    FROM (SELECT * FROM {silver_table})
    FILE_FORMAT = (TYPE = PARQUET COMPRESSION = SNAPPY)
    SINGLE = TRUE
    OVERWRITE = TRUE;
    """
    session.sql(copy_out_sql).collect()
    print(f"✅ Exported {silver_table} -> @silver_csv/{entity}/Full_load/csv_files/{folder_path}{capital_entity}.parquet")

# ---------- Finish ----------
session.close()
print("🔒 Session closed. Full load complete.")
